In [1]:
import sys
sys.path.append("j:/01 Fichier/05 Code/99 - Autre/02 Sujetdex/")
import requests
from bs4 import BeautifulSoup
import os
import re
from PyPDF2 import PdfReader
import importlib
import gestion_database as gd
import Levenshtein
importlib.reload(gd)

Chargement de j:\01 Fichier\05 Code\99 - Autre\02 Sujetdex\gestion_database.py
Chargement de j:\01 Fichier\05 Code\99 - Autre\02 Sujetdex\gestion_database.py


<module 'gestion_database' from 'j:\\01 Fichier\\05 Code\\99 - Autre\\02 Sujetdex\\gestion_database.py'>

In [2]:
def extract_links_from_page(url):
    """
    Extracts all hyperlinks from a webpage.

    Parameters:
        url (str): The URL of the webpage to scrape.

    Returns:
        list: A list of URLs found on the page.
    """
    try:
        # Make an HTTP request to fetch the page content
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> tags with an href attribute
    links = [requests.compat.urljoin(url, link['href']) for link in soup.find_all('a', href=True)]

    return links

visited_links = {}

def fetch_files_from_page(url, output_folder="downloads", file_extensions=("pdf", "doc", "docx"), filter_function=None):
    """
    Fetches all files with specified extensions from a given URL and saves them locally.

    Parameters:
        url (str): The URL of the webpage to scrape.
        output_folder (str): The folder to save the downloaded files. Defaults to "downloads".
        file_extensions (tuple): File extensions to download. Defaults to ("pdf", "doc", "docx").
        filter_function (callable): A function that takes a file name as input and returns True if the file should be downloaded, False otherwise. Defaults to None.

    Returns:
        list: List of downloaded file paths.
    """
    # Dictionary to track visited links and avoid duplicates
    

    # Get all links from the page
    all_links = extract_links_from_page(url)
    file_links = []

    for file_url in all_links:
        # Skip if the link has already been processed
        if file_url in visited_links:
            continue

        # Mark the link as visited
        visited_links[file_url] = True

        # Check file extension
        if file_url.split('.')[-1].lower() in file_extensions:
            file_links.append(file_url)

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    downloaded_files = []

    # Download each file
    for file_url in file_links:
        try:
            file_response = requests.get(file_url, stream=True)
            file_response.raise_for_status()

            # Extract the file name
            file_name = os.path.basename(file_url)

            # Apply the filter function if provided
            if filter_function and not filter_function(file_name):
                print(f"Skipped: {file_name}")
                continue

            output_path = os.path.join(output_folder, file_name)

            # Save the file locally
            with open(output_path, 'wb') as f:
                for chunk in file_response.iter_content(chunk_size=8192):
                    f.write(chunk)

            downloaded_files.append(output_path)
            print(f"Downloaded: {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {file_url}: {e}")

    return downloaded_files

In [3]:
def link_all_files(folder: str) -> list[str]:
    """Fonction qui renvoie tout les fichiers pdf d'un dossier et de ses sous-dossiers
    
    param folder: le chemin du dossier à explorer
    type folder: str
    return: la liste des fichiers pdf
    rtype: list[str]
    """
    linked_files = []

    for root, _, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(root, file)
            linked_files.append(file_path)

    return linked_files

# Fonction en lien avec le PDF

In [4]:
def extract_text_from_pdf(file_path:str) -> list[str]:
    """Fonction qui renvoie le texte du PDF 

    :param file_path: lien du fichier
    :type file_path: str
    """
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pages = pdf_reader.pages
        resultats = [page.extract_text() for page in pages]
        return resultats

In [5]:
def extract_text_of_page_from_pdf(file_path:str, i: int) -> str:
    """Fonction qui renvoie la page spécifiée d'un document pdf.

    :param file_path: lien du fichier
    :type file_path: str
    :param i: numero de la page a extraire (commence avec la page 1)
    :type i: int
    :return: Text de la page
    :rtype: str
    """    
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pages = pdf_reader.pages
        try:
            resultats = pages[i-1].extract_text()
            return resultats
        except IndexError:
            raise Exception("La page demandée n'existe pas")
        
    

In [6]:
def extract_first_page_from_pdf(file_path:str) -> str:
    return extract_text_of_page_from_pdf(file_path, 1)

# Centrale

In [7]:
baselink = "https://www.concours-centrale-supelec.fr/CentraleSupelec"
annee = [i for i in range(1998,2025)]
filiere = ["MP","PC","MPI","PSI","TSI"]
sujet = ["","sujets"]

In [8]:
def supprimer_fichiers_contenant_recursivement(repertoire: str, chaine_a_chercher: str) -> None:
    """Supprime tous les fichiers dans un répertoire et ses sous-répertoires
    dont le nom contient une chaîne de caractères spécifique.

    :param repertoire: Chemin du répertoire où commencer la recherche
    :type repertoire: str
    :param chaine_a_chercher: Chaîne de caractères à chercher dans les noms de fichiers
    :type chaine_a_chercher: str
    :return: Cette fonction ne retourne rien
    :rtype: None
    """
    for root, dirs, files in os.walk(repertoire):
        for file in files:
            if chaine_a_chercher in file:
                chemin_complet = os.path.join(root, file)
                os.remove(chemin_complet)
                print(f"Fichier supprimé : {chemin_complet}")


In [9]:
"""for f in filiere:
    for a in annee:
        for sujets in sujet:
            url = f"{baselink}/{a}/{f}/{sujets}"
            fetch_files_from_page(url, output_folder=f"data/centrale/{f}/{a}", filter_function=filtre)"""

'for f in filiere:\n    for a in annee:\n        for sujets in sujet:\n            url = f"{baselink}/{a}/{f}/{sujets}"\n            fetch_files_from_page(url, output_folder=f"data/centrale/{f}/{a}", filter_function=filtre)'

In [10]:
def supprimer_fichiers_contenant_recursivement(repertoire: str, chaine_a_chercher: str) -> None:
    """Supprime tous les fichiers dans un répertoire et ses sous-répertoires
    dont le nom contient une chaîne de caractères spécifique.

    :param repertoire: Chemin du répertoire où commencer la recherche
    :type repertoire: str
    :param chaine_a_chercher: Chaîne de caractères à rechercher dans les noms de fichiers
    :type chaine_a_chercher: str
    :return: Cette fonction ne retourne rien
    :rtype: None
    """
    for root, dirs, files in os.walk(repertoire):
        for file in files:
            if chaine_a_chercher in file:
                chemin_complet = os.path.join(root, file)
                os.remove(chemin_complet)
                print(f"Fichier supprimé : {chemin_complet}")


In [11]:
"""supprimer_fichiers_contenant_recursivement(repertoire= "data", chaine_a_chercher = "TSI")"""

'supprimer_fichiers_contenant_recursivement(repertoire= "data", chaine_a_chercher = "TSI")'

In [12]:
def extraire_titres_et_questions(fichier_pdf: str) -> list:
    """Extrait les titres et numéros de questions d'un fichier PDF.

    :param fichier_pdf: Chemin vers le fichier PDF
    :type fichier_pdf: str
    :return: Liste des numéros de questions extraites du fichier PDF
    :rtype: list
    """
    with open(fichier_pdf, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pages = pdf_reader.pages

        resultats = []
        for page in pages:
            texte_page = page.extract_text()
            pattern = r"Q\s*(\d+[A-Z]?\d*\)?)\.?\s+(.*)" 
            matches = re.findall(pattern, texte_page, re.MULTILINE)
            
            for match in matches:
                if match:
                    numero_question, titre = match
                    resultats.append((numero_question.strip()))

        return resultats


In [13]:
def get_nom_epreuve(file: str) -> str:
    """Donne le nom de l'épreuve à partir du fichier PDF.

    :param file: Lien du fichier PDF
    :type file: str
    :return: Nom de l'épreuve
    :rtype: str
    """
    page = extract_first_page_from_pdf(file)
    page_couper = page.split("\n")

    for ligne in page_couper:
        ligne = ligne.strip()

        # Ignorer les lignes contenant une date (français, anglais, portugais, format ISO, etc.)
        if re.search(r"(?:\w{3,9} \d{1,2},? \d{4}|\d{4}-\d{2}-\d{2}).*Page ?\d+/\d+", ligne, re.IGNORECASE):
            continue

        # Ignorer les codes du type "S013/2021-11-24"
        if re.match(r"[A-Z]?\d{3,4}/\d{4}-\d{2}-\d{2}", ligne):
            continue
        
        if ligne == " .. " or ligne == "..":
            continue

        # Séparer une année collée au titre (ex: "2011Portugais" → "Portugais")
        ligne = re.sub(r"^\d{4}\s*\.*\s*", "", ligne).strip()

        # Vérifier si la ligne peut être un titre (pas vide, ne commence pas par un chiffre, etc.)
        if ligne and not ligne[0].isdigit() and len(ligne.split()) <= 3:
            return ligne  # On retourne le premier titre valide trouvé

    return ""  # Retourne une chaîne vide si aucun titre n'est trouvé


In [14]:
def get_all_epreuve_name(folder:str)-> set[str]:
    """Renvoie tout les noms d'épreuves

    :param folder: chemin du dossier
    :type folder: str
    :return: tout les noms d'épreuves
    :rtype: set[str]
    """

    if os.listdir(folder) != []:
        liste_results = [extract_first_page_from_pdf(file) for file in link_all_files(folder)]
        liste_cut_results = [result.split("\n") for result in liste_results]
        liste_nom = [result[1] for result in liste_cut_results]
        
        # On enelve l'année devant la matiere
        liste_nv_deb  = [re.sub(r"\d*.? ?","" ,nom[:4]) for nom in liste_nom]
        liste_nv_nom = [deb+nom[4:] for deb,nom in zip(liste_nv_deb,liste_nom)]
        
        for i in range(len(liste_nv_nom)):
            if liste_nv_nom[i] == '..':
                liste_nom[i] = liste_results[i].split("\n")[2]
                liste_nv_deb[i] = re.sub(r"\d\.*\s*.*","" ,liste_nom[i][:4])
                liste_nv_nom[i] = liste_nv_deb[i]+liste_nom[i][4:]
        
        d:set = set()           
        for nom in liste_nv_nom:
            d.add(nom)
            
    return d


In [15]:
#all_epreuve = get_all_epreuve_name(r"data/centrale")

In [16]:
def initialise_liste_epreuve():
    """Parcours toutes les filières du dossier centrale et initialise la liste des epreuves
    Elle met donc a jour la base de donnee database.db
    """
    list_epreuve = []
    for f in filiere:
        all_epreuve = get_all_epreuve_name(r"data/centrale/"+f)
        for epreuve in all_epreuve:
            list_epreuve.append(gd.Epreuve(epreuve, f))

In [17]:
def rename_epreuve(lien_epreuve:str, nv_nom:str) -> None:
    """Renomme une epreuve

    :param lien_epreuve: lien vers le fichier de l'epreuve
    :type lien_epreuve: str
    :param nv_nom: le nouveau nom de l'epreuve
    :type nv_nom: str
    """
    try :os.rename(lien_epreuve, nv_nom)
    except:
        print("Erreur lors du renommage ", lien_epreuve, nv_nom)

In [18]:
liste_matiere = {}
liste_epreuve = {}

In [19]:
def rename_all_epreuve(dossier:str="data/centrale") -> None:
    """Renomme toute les epreves du dossier par nom
    Ainsi l'arborescences des fichiers donne :
    1/ Banque
    2/ Filiere
    3/ Annee
    4/ Epreuve

    :param dossier: lien du dossier ou il faut modifier les noms
    :type dossier: str
    """
        
    for f in filiere:
        liste_matiere[f] = set()
        liste_epreuve[f] = {}
        for a in annee:
            liste_lien = link_all_files(dossier+"/"+f+"/"+str(a)+"/")
            liste_nom = [get_nom_epreuve(file) for file in liste_lien]
            
            #enleve le option au debut et l'ajoute au liste_matiere
            for nom in liste_nom:
                matiere = re.sub(r"Option","",nom)
                try:
                    e = liste_matiere[f].pop()
                    accumulateur = []
                    while Levenshtein.distance(matiere,e)/((len(matiere)*len(e))**(1/2)) > 1:
                        accumulateur.append(e)
                        e = liste_matiere[f].pop()
                        
                    matiere = min(e,matiere)
                    liste_matiere[f].add(matiere)
                    if nom not in liste_matiere[f]:
                        liste_epreuve[f][nom] = gd.Epreuve(nom,f,matiere)
                    else:
                        gd.incr(liste_epreuve[f][nom])
                        gd.modif_matiere(liste_epreuve[f][nom], matiere)
                    
                    for e in accumulateur:
                        liste_matiere[f].add(e)
                except:
                    liste_matiere[f].add(matiere)
                    liste_epreuve[f][nom] = gd.Epreuve(nom,f,matiere)
                
                
                                
            
            for i in range(len(liste_nom)):
                if liste_nom[i] == liste_lien[i]:
                    liste_nom.pop(i)
                    liste_lien.pop(i)
            for i in range(len(liste_nom)):
                try : liste_nom[i] = dossier+"/"+f+"/"+str(a)+"/"+liste_nom[i]+".pdf"
                except : print("Erreur lors de l'envoie du renommage de  : ",liste_lien[i] ,liste_nom[i])
            for lien,nom in zip(liste_lien,liste_nom):
                rename_epreuve(lien, nom)

In [20]:
rename_all_epreuve("data/centrale")
print(liste_matiere, liste_epreuve)

{'MP': set(), 'PC': {'Physique 2'}, 'MPI': set(), 'PSI': {'S2I'}, 'TSI': {'Anglais'}} {'MP': {'Allemand': <gestion_database.Epreuve object at 0x0000021474370140>, 'Arabe': <gestion_database.Epreuve object at 0x0000021473C255B0>, 'Chinois': <gestion_database.Epreuve object at 0x00000214741FB080>, 'Espagnol': <gestion_database.Epreuve object at 0x00000214741496A0>, 'Informatique': <gestion_database.Epreuve object at 0x0000021474391E50>, 'Mathématiques 2': <gestion_database.Epreuve object at 0x000002146E85BA70>, 'Physique Chimie': <gestion_database.Epreuve object at 0x0000021474410C80>, 'Physique–Chimie': <gestion_database.Epreuve object at 0x0000021474410B60>, 'Portugais': <gestion_database.Epreuve object at 0x0000021474349A60>, 'Russe': <gestion_database.Epreuve object at 0x00000214743483B0>, 'Rédaction': <gestion_database.Epreuve object at 0x000002147434B830>, 'S2I': <gestion_database.Epreuve object at 0x00000214741BFAD0>, 'Mathématiques 1': <gestion_database.Epreuve object at 0x000002

In [21]:
mot1 = "Russe"
mot2 = "Rédaction"
Levenshtein.distance(mot1, mot2)/((len(mot1)*len(mot2))**(1/2))

1.1925695879998879

In [22]:
mot1 = "Physique-chimie 1"
mot2 = "Physique"
Levenshtein.distance(mot1, mot2)/((len(mot1)*len(mot2))**(1/2))

0.7717436331412897

In [23]:
mot1 = "Option informatique"
mot2 = "Option S2I"
Levenshtein.distance(mot1, mot2)/((len(mot1)*len(mot2))**(1/2))

0.870571500132014